<a href="https://colab.research.google.com/github/jmolina010/ejemplo_preprocesamiento_ttextos/blob/main/ejemplo_completo_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descripción del problema

Disponemos de un dataset que contiene críticas de 119.003 películas del periodo de 1900 a 2020.
Los atributos del dataset son:

    • Título: film_name
    • Género: genre
    • Puntuación promedio: film_avg_rate
    • Puntuación asociada a la crítica: review_rate
    • Título de la crítica: review_title
    • Crítica: review_text

##Objetivo

Generar un modelo predictivo que genere una puntuación provisional (sugerida) asociada a la crítica, a partir del texto de la misma. Nota: Nos vamos a limitar al preprocesamiento de la data.
Se parte de un dataset ya limpio y válido, que podrás encontrar en el repositorio git, junto al cuaderno del ejercicio. Debido a esto, las tareas básicas de investigación y limpieza de la data, vamos a saltarlas, centrándonos en los pasos que realmente tienen interés desde el punto de vista de la PNL.

#Planteamiento de la solución

El objetivo es, como sabemos, preparar el dataset para que se pueda utilizar en el entrenamiento de un modelo, por lo que a continuación vamos a abordar los siguientes puntos:

    1. Lectura del fichero, identificación y selección de columnas
    2. Lectura de las críticas, tokenización, eliminación de signos de puntuación y stop words, generación de columna nueva y guardado de nueva versión del fichero
    3. Determinación de frecuencia de aparición de tokens, y posible eliminación de tokens menos frecuentes. Nota: Esta tarea, dependiendo del problema, puede abordarse en este momento o, quizá como parte el punto 5, pero debido a la naturaleza didáctica de este caso, y al elevado volumen de tokens se va a abordar, inicialmente, en este punto.
    4. Sustitución de sinónimos de tokens relevantes
    5. Lematización de verbos
    6. Planteamiento de franjas para selección de tokens y selección de tokens por franja
    7. Generación de codificación OneHot

En cada paso, generaremos un fichero csv nuevo con el resultado de las tareas realizadas.

#Lectura del fichero, identificación y selección de columnas

In [ ]:
# importación de librerías y funciones necesarias
import pandas as pd
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
import collections
import operator

!python -m spacy download es_core_news_sm

In [ ]:
data_original = pd.read_csv('/content/reviews_filmaffinity.csv', sep='##',
                            engine='python')
print(data_original.head())

# puede observarse que nos interesan únicamente dos columnas, que serán
# review_rate como variable dependiente y review_text como columna para llevar
# a cabo todo el preprocesamiento

data = pd.DataFrame({'review_text': data_original[data_original.columns[-1]],
                     'review_rate': data_original[data_original.columns[3]]})

print('\n\nDataframe nuevo:')
print('----------------\n\n')
print(data.head())


# inspeccionamos cada una de las dos columnas
print('\n\n')
print(data[data.columns[0]])

print('\n')
print(data[data.columns[1]])

# se observa que disponemos de una versió￳n reducida del dataset, en la que solo
# hay 8603 ocurrencias (filas) en el dataframe, y no las que se indicaban.
# esto no es problema, ya que el tratamiento de los tokens se realiza de
# forma gen￩rica independiente de la cantidad de filas.


In [3]:
# ahora, como la columna objetivo es review rate,se van a analizar nulos
print(f'Reviews rates: {data["review_rate"].unique().tolist()}')
print(f'Criticas nulas: {data["review_text"].isna().sum()}')

# Se observa que hay valoraciones nulas, se eliminan del dataframe
print(f'Filas de data antes de eliminar: {len(data)}')
data.drop(data[data['review_rate'].isna()].index, inplace=True)
print(f'Filas de data después de eliminar: {len(data)}')

Reviews rates: [3.0, 2.0, 8.0, 1.0, 6.0, 4.0, 5.0, 7.0, 9.0, 10.0, nan]
Criticas nulas: 0
Filas de data antes de eliminar: 8603
Filas de data después de eliminar: 8601


In [4]:
# para terminar con este punto, vamos a asegurarnos guardar el nuevo dataframe
# en disco, en formato csv
# en este caso, el separador debe ser un único caracter.

# vamos a asegurarnos de que el tabulador, '\t', no se emplea en las cr￭ticas.
criticas = data.review_text.values
aparece = [True if '\t' in review else False for review in criticas]
aparece = True if True in aparece else False
if aparece:
  print('No se puede separar por tabulador')
else:
  print('El tabulador es un buen separador')

data.to_csv('/content/criticas_columnas_seleccionadas.csv', sep='\t',
            index=False)
print('Dataframe guardado')

El tabulador es un buen separador
Dataframe guardado


# Generación de tokens básicos
###Lectura de las críticas, tokenización, eliminación de signos de puntuación y stop words, generación de columna nueva y guardado de nueva versión del fichero

In [ ]:
data = pd.read_csv('/content/criticas_columnas_seleccionadas.csv', sep='\t')

lista_tokens = [] # guardaremos en esta lista los tokens b￡sicos de cata cr￭tica
nlp=spacy.load('es_core_news_sm')

for fila in range(len(data)):
  # procesamos una critica con Spacy
  critica = nlp(data.iloc[fila]['review_text'])
  if fila<100 and fila % 10 == 0:
    print(f'Procesando fila {fila}')
  if fila>0 and fila % 100 == 0:
    print(f'Procesando fila {fila}')
  # generamos lista de tokens eliminando puntuacion y caracteres especiales,
  # así como stop words básicas
  tokens = [str(token).lower() for token in critica
            if token.is_alpha and token not in STOP_WORDS]

  # lista_tokens es una lista de listas, una nueva columna del DataFrame
  lista_tokens.append(tokens)

data['tokens_basicos'] = lista_tokens

print(data)

In [6]:
data.to_csv('/content/criticas_tokens_basicos.csv', sep='\t',
            index=False)
print('Dataframe guardado')

Dataframe guardado


#Determinación de frecuencia de aparición de tokens

##_y eliminación de tokens menos frecuentes_

In [7]:
data = pd.read_csv('/content/criticas_tokens_basicos.csv', sep='\t')
print(data.columns)

Index(['review_text', 'review_rate', 'tokens_basicos'], dtype='object')


In [8]:
print(data)

                                            review_text  review_rate  \
0     La mayor virtud de esta película es su existen...          3.0   
1     No soy un experto cinéfilo; pero pocas veces m...          2.0   
2     Si no eres un incondicional del humor estilo T...          2.0   
3     No sé qué está pasando; si la gente se deja ll...          2.0   
4     ""Pero cuando amanece";y me quedo solo;siento ...          2.0   
...                                                 ...          ...   
8596  Buena no; lo siguiente. Por fin un film serio ...         10.0   
8597  Me esperaba mucho; pero que mucho; más.Guión m...          3.0   
8598  De mal cuerpo como sensación al finalizar; de ...          4.0   
8599  Los que han añadido comentarios os lo han dich...          1.0   
8600  Fui a ver esta película de cine con entusiasmo...          2.0   

                                         tokens_basicos  
0     ['la', 'mayor', 'virtud', 'de', 'esta', 'pelíc...  
1     ['no', 'soy',

In [9]:
# son muchos tokens, por lo que hay que implementar un proceso optimizado para
# generar el diccionario de apariciones de forma  manual.

dict_tokens = {}

for fila in range(len(data)):
  # se obtiene la lista de tokens como string y se convierte a lista
  tokens_fila = data.iloc[fila]['tokens_basicos'][1:-1].replace("'", "")
  tokens_fila = data.iloc[fila]['tokens_basicos'][1:-1].split(',')
  tokens_fila = [token.strip()[1:-1] for token in tokens_fila]
  for token in tokens_fila:
    if token in dict_tokens:
      dict_tokens[token] += 1
    else:
      dict_tokens[token] = 1

def ordenar_diccionario_tokens(dic: dict) -> dict:
  return dict(sorted(dic.items(), key=operator.itemgetter(1)))

print(dict_tokens)
dict_tokens = ordenar_diccionario_tokens(dict_tokens)
print(dict_tokens)
print(f'Total tokens: {len(dict_tokens)}')




{'la': 66082, 'mayor': 675, 'virtud': 76, 'de': 97030, 'esta': 6969, 'película': 16657, 'es': 28167, 'su': 11552, 'existencia': 64, 'el': 41445, 'hecho': 1855, 'que': 79256, 'podamos': 45, 'jugar': 118, 'con': 20260, 'los': 22847, 'tópicos': 533, 'más': 12819, 'extremos': 41, 'las': 13356, 'identidades': 3, 'patrias': 18, 'andaluza': 32, 'y': 56565, 'vasca': 118, 'sin': 5823, 'nadie': 710, 'se': 20952, 'escandalice': 4, 'ni': 4144, 'ponga': 60, 'grito': 48, 'en': 43093, 'cielo': 151, 'indica': 20, 'mucho': 3151, 'nuestra': 488, 'madurez': 46, 'como': 12199, 'nación': 27, 'pese': 391, 'a': 36448, 'quien': 768, 'bueno': 1283, 'corrijo': 3, 'hacer': 2268, 'mofa': 13, 'befa': 1, 'sobre': 2954, 'vascos': 609, 'nacionalismo': 20, 'vasco': 240, 'del': 14077, 'grado': 59, 'normalización': 5, 'ciertas': 178, 'cuestiones': 55, 'antes': 820, 'eran': 302, 'llagas': 3, 'abiertas': 10, 'siempre': 1885, 'dispuestas': 4, 'sangrar': 1, 'hago': 67, 'corrección': 32, 'porque': 3874, 'andaluces': 145, 'ha

### Se puede observar que existen numerosos tokens con muy pocas apariciones, pongamos en nuestro caso 30 o menos, así como otros tokens que aparecen un elevadísimo número de veces, pongamos por caso, 7000 o más.

Se procede a eliminar todos esos tokens y a revisar el diccionario generado.

In [10]:
print(f'Tamaño del diccionario original: {len(dict_tokens)}')
eliminar = []

for key in dict_tokens.keys():
  if dict_tokens[key]<=30 or dict_tokens[key]>=7000:
    eliminar.append(key)

for key in eliminar:
  del dict_tokens[key]

print(f'Tamaño del diccionario nuevo: {len(dict_tokens)}')

Tamaño del diccionario original: 55832
Tamaño del diccionario nuevo: 4282


###Se puede observar una reducción drástica del número de elementos del diccionario.

Se van a aplicar, por lo tanto, y previo a un análisis manual, otras técnicas de reducción del diciconario, que deberán aplicarse siempre pensando si tienen sentido en cada caso particular.

En nuestro caso, se trata de predecir la puntuación de la película a partir de la crítica, por lo que, además, se van a probar diferentes opciones.


In [11]:
# 1. Lematización. Se van a sustituir verbos por sus infinitivos.


# a. Con todos los token se genera un string que deberá procesarse con spacy
str_tokens = ''
for token in dict_tokens.keys():
  str_tokens += token + ' '
str_tokens = str_tokens[:-1]

# b. Se analizan los lemas
doc = nlp(str_tokens)
print ('Idx'.ljust(6) + 'Token'.ljust(25) + 'PoS'.ljust(7) + 'Lema'.ljust(30))
for idx, token in enumerate (doc):
      if idx<25: # para no generar una salida muy grande...
        print(str(idx).ljust(6) + token.text.ljust(25) + token.pos_.ljust(7) + token.lemma_.ljust(30))

# c. Se reagrupan tokens por su lema.

# Ahora que ya se puede procesar, se trabaja a partir de esta lista de tokens
# del diccionario. Debe observarse que se deben procesar los token que
# tienen como PoS VERB o AUX. Si modificas el limite del 'if' encontrarás
# algún AUX.

# Si de modifican las key de un diccionario, es posible que se intenten duplicar
# claves, lo que generaría una excepción. Por ello swe va a generar un dict
# nuevo a partir de los tokens procesados con Spacy

new_dict_tokens = {}
for idx, token in enumerate (doc):
  if token.pos_ in ['VERB', 'AUX']:
    if token.lemma_ in new_dict_tokens:
      new_dict_tokens[str(token.lemma_)] += 1
    else:
      new_dict_tokens[str(token.lemma_)] = 1
  else:
    new_dict_tokens[str(token)] = dict_tokens[str(token)]


print(f'Tamaño del diccionario anterior: {len(dict_tokens)}')
print(f'Tamaño del diccionario nuevo: {len(new_dict_tokens)}')

dict_tokens = ordenar_diccionario_tokens(new_dict_tokens)
print(f'Nuevo diccionario: {dict_tokens}')


Idx   Token                    PoS    Lema                          
0     ejemplar                 ADJ    ejemplar                      
1     encajan                  VERB   encajar                       
2     cita                     NOUN   cita                          
3     ii                       PROPN  ii                            
4     insulso                  ADJ    insulso                       
5     alabar                   VERB   alabar                        
6     venden                   VERB   vender                        
7     extranjeros              ADJ    extranjero                    
8     llevando                 VERB   llevar                        
9     conviene                 ADJ    conviene                      
10    trascendencia            NOUN   trascendencia                 
11    manía                    VERB   manir                         
12    sketches                 NOUN   sketch                        
13    característica           ADJ

In [12]:
# 2. Eliminar los token demasiado cortos, por ejemplo de longitud 6.

# Primero se van a extraer del diccionario y mostrarlos en una lista
lista_cortos = []
for token in dict_tokens.keys():
  if len(token)<=6: lista_cortos.append(token)

print(lista_cortos)
print(f'Total: {len(lista_cortos)}')

# esta lista debería repasarse manualmente, y eliminar de ella aquellos términos
# que pueden ser relevantes al puntuar, como por ejemplo 'plagio', 'pesada',
# 'aburre', 'atrapa', 'dramón', 'chulo', 'placer', 'épico', 'vilo' u otros.
# se han elegido éstos en un repaso básico de la lista. Los tokens a mantener
# dependen del contexto y del objetivo del problema a resolver, por supuesto
eliminar = ['plagio', 'pesada', 'aburre', 'atrapa', 'dramón', 'chulo',
            'placer', 'épico', 'vilo']
for elim in eliminar:
  if elim in lista_cortos:
    lista_cortos.remove(elim)

print('eliminados los importantes...')
print(f'Total: {len(lista_cortos)}')

# cuando ya se tiene la seguridad de que estos token pueden eliminarse, se
# eliminan del diccionario

for token in lista_cortos:
  if token in dict_tokens:
    del dict_tokens[token]

dict_tokens = ordenar_diccionario_tokens(dict_tokens)
print(f'Nuevo diccionario: {dict_tokens}')
print(f'Total: {len(dict_tokens)}')

# dict_tokens, de poco más de 2000 elementos, ya se podría repasar manualmente,
# eliminando aquellos que son irrelevantes en el contexto de la valoracion de
# una película, si bien no se va a proceder con esta tarea por los fines
# didácticos del ejercicio

# Otra tarea a realizar es la reagrupación de tokens sinónimos en un solo token.


['alabar', 'manir', 'valar', 'montar', 'frar', 'robo', 'mover', 'dramón', 'dotar', 'notar', 'bajar', 'pintir', 'oliver', 'sofá', 'andar', 'deír', 'atraer', 'huir', 'asomo', 'opinar', 'causar', 'surgir', 'debeer', 'raya', 'rozar', 'oir', 'vuelo', 'grabar', 'tir', 'volar', 'beber', 'cantar', 'lucir', 'bordir', 'quedé', 'cerrar', 'cuco', 'ruiz', 'ironir', 'luchar', 'mandar', 'parir', 'copiar', 'yon', 'unax', 'súper', 'tostón', 'frikis', 'gir', 'aída', 'soltar', 'restar', 'sitúar', 'abusar', 'véar', 'meten', 'llenar', 'jo él', 'doler', 'óscar', 'traer', 'spain', 'decair', 'matar', 'tebeo', 'captar', 'sentar', 'pd', 'elegir', 'airbag', 'playa', 'alejar', 'belir', 'pixar', 'comer', 'precio', 'iv', 'juzgar', 'ester', 'ser él', 'oler', 'negar', 'sumar', 'parar', 'díaz', 'vean', 'rivera', 'ocupar', 'probar', 'comar', 'joder', 'chaval', 'echar', 'jaumir', 'durar', 'pagar', 'ayalar', 'salvan', 'juicio', 'sonar', 'salí', 'rodear', 'lujo', 'poseer', 'aburre', 'motín', 'dosis', 'cortar', 'meter', 'e

##Planteamiento de franjas para selección de tokens y selección de tokens por franja

###Una vez que se dispone de un diccionario de tokens limitado, se debe proceder a identificar aquellos que aparecen con mayor frecuencia en grupos de valoraciones.

Por ello se identifican distintas franjas de puntuación:

  * Grupo 1: 0 - 2.5
  * Grupo 2: 2.6 - 5
  * Grupo 3: 5.1 - 7.5
  * Grupo 4: 7.6 - 10


 y se calcula las veces que se repite cada token en cada una de las franjas.

In [13]:
# Se lee el dataset original
data = pd.read_csv('/content/criticas_columnas_seleccionadas.csv', sep='\t')

# se genera un dataframe por cada grupo.
data_1 = data.loc[(data['review_rate'] >=0) & (data['review_rate'] <=2.5)]
print(data_1)

data_2 = data.loc[(data['review_rate'] >=2.6) & (data['review_rate'] <=5)]
print(data_2)

data_3 = data.loc[(data['review_rate'] >=5.1) & (data['review_rate'] <=7.5)]
print(data_3)

data_4 = data.loc[(data['review_rate'] >=7.6) & (data['review_rate'] <=10)]
print(data_4)

print(f'Total data: {len(data)}')
print (f'Total Grupo 1: {len(data_1)}')
print (f'Total Grupo 2: {len(data_2)}')
print (f'Total Grupo 3: {len(data_3)}')
print (f'Total Grupo 4: {len(data_4)}')
print (f'Suma de Grupos: {len(data_1)+len(data_2)+len(data_3)+len(data_4)}')
print(f'Reviews rates: {data["review_rate"].unique().tolist()}')

# Debido a que no se va a implementar el modelo en este ejemplo, no se van a
# abordar determinados aspectos, tales como el posible balanceo o desbalanceo
# del dataset -lo cual no tiene nada que ver con el tamaño de cada grupo,
# aunque los grupos si que influyen en los tokens-, aspecto que siempre hay que
# tener en consideración.

# Este ejercicio se centra en las diferentes técnicas de preprocesaqmiento,
# tokenización y preparación de un dataset textual.


                                            review_text  review_rate
1     No soy un experto cinéfilo; pero pocas veces m...          2.0
2     Si no eres un incondicional del humor estilo T...          2.0
3     No sé qué está pasando; si la gente se deja ll...          2.0
4     ""Pero cuando amanece";y me quedo solo;siento ...          2.0
8     Puedo entender que Torrente I y II y Lo imposi...          1.0
...                                                 ...          ...
8312  No me gusta lo que cuenta y no veo nada intere...          1.0
8320  Estoy más que harto de que los críticos españo...          2.0
8595  Esto es lo que consideramos bueno?Pagar una en...          1.0
8599  Los que han añadido comentarios os lo han dich...          1.0
8600  Fui a ver esta película de cine con entusiasmo...          2.0

[1162 rows x 2 columns]
                                            review_text  review_rate
0     La mayor virtud de esta película es su existen...          3.0
6     El 

In [14]:
# con cada grupo se debe generar un diccionario de tokens
# se debe generar, por lo tanto, una función que preprocese las criticas de un
# dataframe y devuelva el diccionario con el recuento de las apariciones de
# cada token en ese grupo (dataframe)
# pero cuidado, se hace partiendo de los tokens del diccionario general, y no
# desde cero

def contar_tokens(data: pd.DataFrame, dict_tokens: dict) -> dict:
  """
  A partir del dataframe recibido y del diccionario de tokens, genera un
  diccionario nuevo donde se indica el numero de apariciones de cada token del
  diccionario original en el dataframe.

  :param data: DataFrame del que se deben tokenizar criticas y contar tokens
  :param dict_tokens: dict con los tokens que se deben tener en cuenta
  :return: dict con las apariciones de cada key de dict_tokens en data
  """

  lista_tokens = []
  nlp=spacy.load('es_core_news_sm')

  tokens_relevantes = [str(token) for token in dict_tokens.keys()]
  for fila in range(len(data)):
    # procesamos una critica con Spacy
    critica = nlp(data.iloc[fila]['review_text'])
    if fila>0 and fila % 200 == 0:
      print(f'Procesando fila {fila}')
    # generamos lista de tokens eliminando puntuacion, caracteres especiales,
    # así como stop words básicas, pero solo teniendo en cuenta los token que
    # nos interesan
    tokens = [str(token).lower() for token in critica
              if token.is_alpha and token not in STOP_WORDS and
              str(token) in tokens_relevantes]

    # en lista tokens guardamos todos los tokens de las criticas del dataframe
    lista_tokens.extend(tokens)

  # ya se dispone de los tokens de todas las criticas, en lista_tokens
  # ahora se genera diccionario para los tokens existentes en dict_tokens

  new_dict_tokens = {}

  for token in lista_tokens:
    if token in tokens_relevantes: # si es un token seleccionado...
      if token in new_dict_tokens:
        new_dict_tokens[token] += 1
      else:
        new_dict_tokens[token] =1

  return ordenar_diccionario_tokens(new_dict_tokens)


In [ ]:
# ahora se genera un diccionario de tokens para cada grupo
print('Generando diccionario de grupo 1')
dict_tokens_1 = contar_tokens(data=data_1.copy(), dict_tokens=dict_tokens)
print('Generando diccionario de grupo 2')
dict_tokens_2 = contar_tokens(data=data_2.copy(), dict_tokens=dict_tokens)
print('Generando diccionario de grupo 3')
dict_tokens_3 = contar_tokens(data=data_3.copy(), dict_tokens=dict_tokens)
print('Generando diccionario de grupo 4')
dict_tokens_4 = contar_tokens(data=data_4.copy(), dict_tokens=dict_tokens)
print('===========================================\n')

print(f'Grupo 1: {len(dict_tokens_1)} elementos.')
print(dict_tokens_1)
print(f'Grupo 2: {len(dict_tokens_2)} elementos.')
print(dict_tokens_2)
print(f'Grupo 3: {len(dict_tokens_3)} elementos.')
print(dict_tokens_3)
print(f'Grupo 4: {len(dict_tokens_4)} elementos.')
print(dict_tokens_4)

# Ahora se debe crear una función que genere un pipeline de preprocesamiento
# de las criticas y que sirva para

In [16]:
# Ahora deberán filtrarse los diccionarios de cada grupo.
# En nuestro ejemplo el dataset quiere orientarse a un ejercicio de regresión
# (podría ser, por ejemplo clasificación, lo cual podría modificar los
# criterios), por lo que de cada diccionario se eliminarán los token que menos
# aparecen y los que más aparecen -recordemos que ya está lematizado el
# diccionario original- y luego se eliminarán de cada diccionario aquellos
# tokens que menos aportan, contextualmente, teniendo en cuenta el significado
# de cada grupo, intentando que la longitud de cada diccionario sea,
# aproximadamente la misma y se trate de, en total, un número reducido de tokens

# primero se van a generar dos métodos
def eliminar_tokens_por_aparicion(lim_inf: int, lim_sup: int, dic: dict) -> dict:

  new_dic = {}

  for t in dic:
    if lim_inf<=dic[t]<=lim_sup: new_dic[t] = dic[t]

  return new_dic

def eliminar_lista_tokens(eliminar: list, dic: dict) -> dict:
  for token in eliminar:
    if token in dic:
      del dic[token]
  return dic


In [17]:
# en primer lugar, analizando los diccionarios de cada grupo, se fijan límites
# inferiores y superiores por grupo:
# 1. 50 - 175
# 2. 75 - 250
# 3. 75 - 250
# 4. 75 - 250

dict_tokens_11 = eliminar_tokens_por_aparicion(50, 175, dict_tokens_1)
print(f'Grupo 1: {len(dict_tokens_11)} elementos.')
print(f'{dict_tokens_11}\n')

dict_tokens_22 = eliminar_tokens_por_aparicion(75, 250, dict_tokens_2)
print(f'Grupo 2: {len(dict_tokens_22)} elementos.')
print(f'{dict_tokens_22}\n')

dict_tokens_33 = eliminar_tokens_por_aparicion(75, 250, dict_tokens_3)
print(f'Grupo 3: {len(dict_tokens_33)} elementos.')
print(f'{dict_tokens_33}\n')

dict_tokens_44 = eliminar_tokens_por_aparicion(75, 250, dict_tokens_4)
print(f'Grupo 4: {len(dict_tokens_44)} elementos.')
print(f'{dict_tokens_44}\n')

# se observa que se ha reducido drásticamente el numero de tokens.
# en este proceso de eliminación debería haberse aplicado alguna regla o
# criterio objetivo de selección matemáticamente justificable, teniendo en
# cuenta la representatividad de cada token respecto al número de críticas y al
# total de apariciones de tokens, si bien por simplificar el proceso e ilustrar
# la técnica, no se ha realizado así.

Grupo 1: 73 elementos.
{'algunas': 50, 'presupuesto': 51, 'simplemente': 51, 'absurdo': 51, 'lamentable': 51, 'mientras': 52, 'imposible': 53, 'mejores': 53, 'interpretación': 53, 'posible': 53, 'supuesto': 54, 'gracias': 54, 'semejante': 55, 'dirección': 55, 'mayoría': 56, 'definitiva': 56, 'general': 56, 'pantalla': 57, 'resultado': 58, 'diálogos': 59, 'apellidos': 60, 'grandes': 60, 'taquilla': 60, 'opinión': 60, 'interpretaciones': 61, 'trabajo': 63, 'entrega': 63, 'sensación': 64, 'producto': 66, 'personas': 66, 'españoles': 66, 'totalmente': 66, 'realmente': 68, 'actuaciones': 68, 'original': 68, 'ejemplo': 70, 'actuación': 71, 'principio': 75, 'siquiera': 76, 'demasiado': 76, 'nuestro': 77, 'durante': 77, 'críticas': 79, 'tópicos': 81, 'tampoco': 81, 'calidad': 82, 'realidad': 82, 'protagonista': 83, 'segunda': 84, 'momentos': 85, 'aburrida': 89, 'situaciones': 93, 'algunos': 94, 'ninguna': 94, 'incluso': 96, 'vergüenza': 99, 'alguien': 99, 'argumento': 110, 'después': 112, 'chi

In [18]:
# finalmente, y aplicando un criterio subjetivo de reducción de los
# diccionarios, se eliminarán de cada diccionario aquellos tokens que,
# aparentemente, no tienen aportaciónj contextual al grupo de valoración
# es decir, se eliminarán de cada diccionario tokens que no aportan
# contenido conexttual a la crítica en función del grupo al que ésta pertenece

# Cabe destacar que es una práctica subjetiva y que requiere de un conocimiento
# específico elevado del problema y de su contexto, y que, en el caso de este
# ejemplo, el componente aleatorio de la selección de tokens es elevado.

eliminar_1 = ['algunas', 'presupuesto', 'simplemente', 'absurdo', 'mientras', 'imposible', 'mejores', 'interpretación', 'posible', 'supuesto', 'gracias', 'semejante', 'dirección', 'mayoría', 'definitiva', 'general', 'pantalla', 'resultado', 'diálogos', 'apellidos', 'grandes', 'taquilla', 'calidad', 'momentos', 'aburrida', 'situaciones', 'algunos', 'ninguna', 'incluso', 'alguien', 'argumento', 'después']
eliminar_2 = ['precisamente', 'esperar', 'tsunami', 'detalles', 'personal', 'españolas', 'primero', 'parecer', 'vestuario', 'absoluto', 'proyecto', 'ciertos', 'televisión', 'correcta', 'muestra', 'especiales', 'directores', 'decepción', 'conjunto', 'perdido', 'destacar', 'reconocer', 'nuestra', 'entender', 'montaje', 'primeros', 'ocasión', 'excelente', 'supuesto', 'realidad', 'diálogos', 'grandes', 'situaciones', 'embargo', 'chistes', 'algunas', 'sentido', 'argumento', 'realmente', 'original', 'nuestro', 'después', 'cualquier', 'protagonista', 'minutos', 'incluso', 'trabajo']
eliminar_3 = ['pequeños', 'respecto', 'espectacular', 'necesario', 'personalidad', 'aquellos', 'habitual', 'presente', 'montaje', 'corazón', 'histórica', 'cinematográfico', 'entonces', 'sonrisa', 'dirigida', 'previsible', 'actrices', 'narración', 'ejemplo', 'actuaciones', 'producción', 'actuación', 'personas', 'tensión', 'resultado', 'animación', 'familia', 'pantalla', 'dirección', 'críticas', 'minutos', 'principio', 'protagonistas', 'sensación', 'ambientación', 'después', 'situaciones', 'argumento']
eliminar_4 = ['problema', 'imaginación', 'magnífico', 'entrega', 'detalle', 'maravilla', 'policía', 'objetivo', 'precisamente', 'críticos', 'ciencia', 'carcelario', 'marismas', 'universo', 'palabras', 'pequeño', 'nacional', 'mostrar', 'sobretodo', 'carrera', 'entonces', 'encontrar', 'atmósfera', 'prejuicios', 'relación', 'nuestros', 'personal', 'primeros', 'montaje', 'laberinto', 'parecer', 'elementos', 'desarrollo', 'fantasía', 'general', 'protagonistas', 'opinión', 'imposible', 'embargo', 'minutos', 'ambientación', 'sentido', 'algunas', 'críticas', 'actuación', 'público', 'efectos', 'muestra', 'actuaciones', 'familia', 'pantalla', 'realmente', 'cualquier', 'argumento']

dict_tokens_111 = eliminar_lista_tokens(eliminar_1, dict_tokens_11)
dict_tokens_222 = eliminar_lista_tokens(eliminar_2, dict_tokens_22)
dict_tokens_333 = eliminar_lista_tokens(eliminar_3, dict_tokens_33)
dict_tokens_444 = eliminar_lista_tokens(eliminar_4, dict_tokens_44)

print(f'Grupo 1: {len(dict_tokens_111)} elementos.')
print(f'{dict_tokens_111}\n')

print(f'Grupo 2: {len(dict_tokens_222)} elementos.')
print(f'{dict_tokens_222}\n')

print(f'Grupo 3: {len(dict_tokens_333)} elementos.')
print(f'{dict_tokens_333}\n')

print(f'Grupo 4: {len(dict_tokens_444)} elementos.')
print(f'{dict_tokens_444}\n')


# ahora se fusionan las claves de los diccionarios en una sola lista
lista_tokens = [str(token) for token in dict_tokens_111.keys()]
lista_tokens.extend([str(token) for token in dict_tokens_222.keys()])
lista_tokens.extend([str(token) for token in dict_tokens_333.keys()])
lista_tokens.extend([str(token) for token in dict_tokens_333.keys()])

lista_tokens = list(set(lista_tokens))

print(f'Total tokens: {len(lista_tokens)}')
print(lista_tokens)

Grupo 1: 41 elementos.
{'lamentable': 51, 'opinión': 60, 'interpretaciones': 61, 'trabajo': 63, 'entrega': 63, 'sensación': 64, 'producto': 66, 'personas': 66, 'españoles': 66, 'totalmente': 66, 'realmente': 68, 'actuaciones': 68, 'original': 68, 'ejemplo': 70, 'actuación': 71, 'principio': 75, 'siquiera': 76, 'demasiado': 76, 'nuestro': 77, 'durante': 77, 'críticas': 79, 'tópicos': 81, 'tampoco': 81, 'realidad': 82, 'protagonista': 83, 'segunda': 84, 'vergüenza': 99, 'chistes': 117, 'público': 122, 'bastante': 123, 'escenas': 126, 'espectador': 128, 'cualquier': 130, 'comedia': 133, 'crítica': 133, 'minutos': 141, 'sentido': 143, 'momento': 151, 'también': 156, 'española': 168, 'siempre': 171}

Grupo 2: 69 elementos.
{'importante': 88, 'situación': 88, 'intento': 88, 'conseguido': 88, 'posible': 89, 'tercera': 89, 'anteriores': 90, 'difícil': 91, 'absolutamente': 94, 'anterior': 95, 'ocasiones': 98, 'talento': 99, 'actuaciones': 107, 'historias': 108, 'metraje': 109, 'secundarios': 11

In [34]:

# se desarrolla una función que genera una lista básica de tokens
# para un texto -crítica- y lo agrega como columna al df
def agregar_tokens(data: pd.DataFrame) -> pd.DataFrame:
  """
  Agrega al df recibido una columna con los tokens basicos de cada crítica

  :param data: DataFrame del que se deben tokenizar criticas y contar tokens
  :return: df con la columna de tokens como lista de strings para cada fila
  """

  nlp=spacy.load('es_core_news_sm')
  listas_tokens_data = []

  for fila in range(len(data)):
    # procesamos una critica con Spacy
    critica = nlp(data.iloc[fila]['review_text'])
    if fila>0 and fila % 200 == 0:
      print(f'Procesando fila {fila}')
    # generamos lista de tokens eliminando puntuacion, caracteres especiales,
    # así como stop words básicas, pero solo teniendo en cuenta los token que
    # nos interesan
    tokens = [str(token).lower() for token in critica
              if token.is_alpha and token not in STOP_WORDS]

    # en lista tokens guardamos todos los tokens de las criticas del dataframe
    listas_tokens_data.append(tokens)

  # agregamos la columna de las listas de tokens
  data['tokens']  =  listas_tokens_data
  return data.copy()



In [ ]:
# ahora se generan los token basicos de cada grupo.

# generamos un unico df con todos los grupos
data = pd.concat([data_1, data_2, data_3, data_4], axis=0)
print(data)

# agregamos los token
data = agregar_tokens(data=data)

print()
print(data)


In [51]:
# una vez que se dispone de un dataframe con los token, hay que iterar en este
# dataframe cada fila, procesando los tokens y, generando una fila del dataframe
# de los datos de entrenamiento
def generar_df_train(data: pd.DataFrame) -> pd.DataFrame:

  dict_df_one_hot = {token: [] for token in lista_tokens}
  dict_df_one_hot['review_rate']= []

  # ahora que ya se dispone de la estructura del diccionario para crear un df
  # se debe rellenar
  # Para ello se procesa el dataframe inicial (se extraen los token de cada
  # crítica) y se identifica si es un token existente en nuestro dataframe de
  # entrenamiento, en su caso, se asigna un 1. También se asigna el review_rate
  for fila in range(len(data)):
    tokens = data.loc[fila]['tokens']

    # para cada columna del df de train, miramos si esta en los token de la fila
    for token in lista_tokens:
      if token in tokens: dict_df_one_hot[token].append(1)
      else: dict_df_one_hot[token].append(0)

    # se agrega review_rate
    dict_df_one_hot['review_rate'].append(data.loc[fila]['review_rate'])

  df_one_hot = pd.DataFrame(dict_df_one_hot)
  return df_one_hot

In [53]:
# se genera el dataframe con los datos de entrenamiento
df_entrenamiento = generar_df_train(data=data)
print(df_entrenamiento)

      actuaciones  todavía  diálogos  momento  previsible  taquilla  \
0               0        0         0        0           0         0   
1               0        1         0        0           0         0   
2               0        0         0        0           0         0   
3               0        0         0        1           0         0   
4               0        0         0        0           0         0   
...           ...      ...       ...      ...         ...       ...   
8596            0        0         0        0           0         0   
8597            0        0         0        0           1         0   
8598            0        0         0        0           0         0   
8599            1        0         0        0           0         0   
8600            0        0         0        0           0         0   

      resultado  animación  nuestro  totalmente  ...  tampoco  apellidos  \
0             0          0        1           1  ...        0          

In [54]:
# se guarda en un CSV
df_entrenamiento.to_csv('/content/dataset_train_test_filmaffinity.csv', sep='\t',
            index=False)
print('Dataframe guardado')

Dataframe guardado
